# Pandas

Структура занятия:

1) Введение
2) Типы данных
3) Операции над данными
4) Агрегирование и группировка

## Введение

[Pandas](https://pandas.pydata.org/docs/) — программная библиотека на языке Python для обработки и анализа данных. Работа pandas с данными строится поверх библиотеки NumPy. 

Основные наборы данных Pandas типов DataFrame и Series применяются в качестве входных в большинстве модулей анализа данных и машинного обучения

Массивы NumPy представляют множество возможностей для работы с полными данными в задачах численных вычислений, однако имеют ограничения при работе с неполными (часть значений отсутствует), неподготовленными (разные типы или единицы изменрения), немаркированными данными.

In [ ]:
pip install pandas

In [ ]:
import pandas as pd
import numpy as np

## Типы данных

На начальном уровне можно считать объекты Pandas расширенной версией структурированных массивов NumPy. Основные структуры Pandas:
- Series - одномерный массив индексированных данных
- DataFrame - обобщённый массив индексированных данных
- Index - массив индексов

**Series** может быть создан из списка

In [ ]:
s = pd.Series([1, 2, 3, 4.0])
s

In [ ]:
s.values  # массив NumPy

In [ ]:
s.index  # массив pd.Index

In [ ]:
s[1]

In [ ]:
s[1:3]

За счёт связи с явным индексом, объек Series значительно гибче нежели массив NumPy. Индекс Series может состоять из значений любого типа и не обязательно должен быть последовательным

In [ ]:
s = pd.Series([1, 2, 3, '4'], index=['f', 's', 't', 'f'])
s

In [ ]:
s['f']

Таким образом, структура чем-то напоминает словарь Python (за исключением, возможности задать повторяющиеся ключи в словаре), более того, Series может быть напрямую создан из словаря

In [ ]:
capitals = pd.Series({'Russia': 'Moscow', 'China': 'Beijing'})
capitals

**DataFrame** может быть рассморен как обобщённый (n-мерный) массив NumPy

In [ ]:
population = pd.Series({'Russia': 144_000_000, 'China': 1_440_000_000})
countries = pd.DataFrame({'population': population, 'capitals': capitals})
countries

DataFrame содержит множество атрибутов, рассмотрим некоторые из них

In [ ]:
# метки столбцов
countries.columns

In [ ]:
# метки индекса или строк
countries.index

In [ ]:
# элементы
countries.values

In [ ]:
# тоже что и индексы
countries.keys()

Обратим внимание на то что 'доступ по ключу' осуществляется не к строкам данных (к конкретным элементам), а к столбцам! Таким образом, можно рассматривать объект типа DataFrame как специализированный словарь.

In [ ]:
countries['population']

In [ ]:
countries['Russia']

Объект DataFrame может быть создан из одного объекта Series, из списка словарей, из словаря объектов Series, из 2х мерного массива NumPy или из структурированного массива NumPy

**Index** - неизменяемый массив индексов, очень похожий по функционалу на массив NumPy, с тем отличием, что индексы невозможно модифицировать стандартными способами. Неизменяемость служит безопасному использованию индекса, его жёсткой привязке к объектам DataFrame.

In [ ]:
towns = pd.read_csv('towns.csv', index_col='city')  # возможно мы потеряли какие-то города =(

In [ ]:
towns

Давайте сразу удалим колонки, которые нам не нужны

In [ ]:
towns[3:7]

In [ ]:
towns[towns['population'] >= 1000]

In [ ]:
towns.region_name

In [ ]:
towns.sort_values('population', ascending=False)

In [ ]:
towns.T

Существует 2 вида индексаторов:
- `.loc` - индексация и срезы с использованием явного индекса
- `.iloc` - с использованием неявного индекса в стиле Python

In [ ]:
towns.loc[:'Москва', :'region_name']

In [ ]:
towns.iloc[:413, :4]

In [ ]:
towns.loc[(towns.population > 1000) & (towns.population < 2000), ['region_name', 'federal_district']]

## Операции над данными

Как и в случае с NumPy, Pandas позволяет оперировать над данными с помощью универсальныйх функций. При использовании унарных функций (изменение знака, тригонометрия) индексы будут сохранены, а при использовании бинарных - Pandas будет автоматически выравнивать индексы. 

In [ ]:
A = pd.DataFrame([[1,2,0],[1,2,3],[1,2,3]])
B = pd.DataFrame([[1,2,0],[1,2],[1,2,]])
C = A + B  # срабатывает выравнивание индекса
C

Обратите внимание на то, как Pandas работает с отсутствующими данными, в данном случае он просто заменяет их на NaN, что в общем случае эквивалентно `None`

Что можно сделать с пустыми значениями:
- выявить `.isnull()`, `.notnull()`
- удалить `.dropna`
- заполнить `.fillna`

In [ ]:
C.notnull()

In [ ]:
C.dropna()

In [ ]:
C.dropna(axis=1)

In [ ]:
C.dropna(how='any')  # 'all'

In [ ]:
C.fillna(0)

Pandas позволяет управлять индексами в уже готовых объектах, например, можно выставить простой инкрементальный индекс и вернуть текущий в колонку данных 

In [ ]:
towns = towns.reset_index()
towns

При помощи индексов можно задавать для данных дополнительные измерения. Создадим **мультииндекс**, при этом, порядок выбираем от общего к частному... это необязательно, порядок может быть любым, и будет зависеть от желаемой группировки

In [ ]:
towns = towns.set_index(['federal_district', 'region_name', 'city'])
towns

Теперь можно работать с данными, используя составной индекс

In [ ]:
towns.loc[('Сибирский', 'Алтайский край', 'Барнаул'), :]

In [ ]:
towns.loc[('Сибирский', 'Алтайский край'), :]

In [ ]:
towns.loc[('Сибирский'), :]

In [ ]:
towns = towns.reset_index()

Одно из выжнейших свойств Pandas - её высокопроизводительные операции соединения и слияния данных. Основной интерфейс `pd.merge`, реализует множество типов соединений: один-к-одному / многие-к-одному / многие-ко-многим. Тип будет зависеть от входных данных: слияние может быть произведено по колонкам с одинаковыми именами, или по указанным вручную колонкам (с импользованием аргументов `on`, `left_on`, `right_on`), или даже по индексам (`left_index`, `right_index`). 

Более продвинутая работа со слиянием напоминает работу с реляционными данными, можно также задать метод соединения при помощи аргумента `how`, возможные варианты: `inner`, `outer`, `left`, `right`

Создадим более сложный датафрейм данных, обогатив изначальные данные по городам информацией по регионам

In [ ]:
regions = pd.read_csv('regions.csv', index_col='Субъект', delimiter=';')

In [ ]:
regions

In [ ]:
regions['region_name'] = regions.index  # создадим колонку с именем, как в датафрейме с городами 

In [ ]:
regions

In [ ]:
regions_s = set(regions['region_name'].unique())

In [ ]:
towns_s = set(towns['region_name'].unique())

In [ ]:
regions_s.difference(towns_s)

In [ ]:
towns_s.difference(regions_s)

In [ ]:
regions.loc['Кабардино-Балкария', 'region_name'] = 'Кабардино-Балкарская Республика'
regions.loc['Карачаево-Черкесия', 'region_name'] = 'Карачаево-Черкесская Республика'
regions.loc['Кемеровская область', 'region_name'] = 'Кемеровская область - Кузбасс'
regions.loc['Тюменская область с ХМАО и ЯНАО', 'region_name'] = 'Тюменская область'
regions.loc['Архангельская область включая НАО', 'region_name'] = 'Архангельская область'

In [ ]:
regions.rename({'округ': 'federal_district', 'площадь': 'region_area', 'процент': 'region_size_percent'}, axis=1, inplace=True)

In [ ]:
regions

In [ ]:
towns_extended = pd.merge(towns, regions, how='left')  # объединим датафреймы по колонкам с одинковыми названиями

In [ ]:
towns_extended

Так как мы выбрали левое соедиенение, данные в towns не пропали, несмотря на то что им не нашлось соответствия из regions. Все поля из regions были заменены на NaN, найдём их по одному из полей

In [ ]:
towns_extended[towns_extended.region_area.isnull()]

## Агрегирование и группировка

Важная часть анализа данных - эффективное обощение: вычисление сводных показателей, в которых одно число позволяет понять природу всего набора данных

Установим пакет `seaborn`, который содержит множество интересных наборов данных

https://science.nasa.gov/exoplanets/exoplanet-catalog/

In [ ]:
pip install seaborn

In [ ]:
import seaborn as sn

In [ ]:
planets = sn.load_dataset('planets')

In [ ]:
planets.shape

In [ ]:
planets.sort_values('year')

Метод `.describe` вычисляет сразу несколько самых используемых сводных показателей для каждого столбца

In [ ]:
planets.describe()

Объекты Pandas имеют набор простых агрегирующих методов, таких как `.count()`, `.mean()`, `.min()`, `.max()`, `.std()`, `.sum()`, `.prod()`. 

Более глубокое исследование обобщённых данных достигается при помощи операции `groupby`, которая производит разбиение (по ключу), применение (какой-либо функции) и объединение (результатов)

In [ ]:
planets.groupby('method')['orbital_period'].mean()

In [ ]:
planets.groupby('method')['orbital_period'].describe()

Можно создавать сводные таблицы (обыно на многомерном индексе)

In [ ]:
planets.pivot_table('number', index='year', columns='method', aggfunc='count')

Мощным средством группировок являются временные ряды. Для того чтобы их использвать, необходимо привести данные к форматц даты и времени.

In [ ]:
planets['date'] = pd.to_datetime(planets['year'], format='%Y')

In [ ]:
planets

In [ ]:
planets_copy = planets.copy()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
planets_pt = planets_copy.pivot_table('number', index='date', columns='method', aggfunc='count')
planets_pt.plot()

In [ ]:
fig, ax = plt.subplots(2, sharex=True)
planets_pt.asfreq('2A', method='ffill').plot(ax=ax[0], style='--.')  # 'A' - код годовой периодичности
planets_pt.asfreq('2A', method='bfill').plot(ax=ax[1], style='-.')
ax[1].legend(['back-fill'])
ax[0].legend(['front-fill'])

## Визуализация

In [ ]:
towns_extended

In [ ]:
plt.scatter('lon', 'lat', data=towns_extended, s='population', alpha=0.2)